In [2]:
import pandas as pd

df = pd.read_csv('../data/occup_df.csv')
df

,Occupation,Code
0,General and Operations Managers,11-1021.00
1,Marketing Managers,11-2021.00
2,Sales Managers,11-2022.00
3,"Financial Managers, Branch or Department",11-3031.02
4,Management Analysts,13-1111.00
5,Financial Analysts,13-2051.00
6,Computer Systems Analysts,15-1121.00
7,Information Security Analysts,15-1122.00
8,"Software Developers, Applications",15-1132.00
9,"Software Developers, Systems Software",15-1133.00


In [4]:
df_train = pd.read_csv('../data/train_df.csv')
df_train

,Title,Code
0,senior program analyst navy strike fighter sen...,15-1121.00
1,senior intelligence analyst iii job,33-3021.06
2,retail wireless sales consultant part,41-2031.00
3,test automation engineer w selenium and gerkin...,15-1121.00
4,public sector portfolio management senior asso...,11-1021.00
...,...,...
12093,tier help desk technical support rep,33-3021.06
12094,road technician mechanic,49-3023.02
12095,health care policy editor,41-2031.00
12096,cyberecurity engineer senior,15-1122.00


In [6]:
df_train.describe()

,Title,Code
count,12098,12098
unique,12098,21
top,senior program analyst navy strike fighter sen...,29-1141.00
freq,1,1186


In [18]:
df2 = df_train[['Code']]
df2.value_counts()

Code      
29-1141.00    1186
15-1142.00    1082
15-1132.00    1080
15-1122.00    1051
15-1133.00     985
11-2021.00     778
13-1111.00     690
33-3021.06     590
11-1021.00     589
11-2022.00     477
15-1121.00     438
41-2031.00     434
11-3031.02     432
43-4051.00     371
15-1151.00     355
49-9071.00     314
13-2051.00     287
31-1014.00     283
53-3032.00     248
49-3023.02     232
15-1134.00     196
dtype: int64